In [1]:
!pip install celluloid

In [2]:
%matplotlib notebook
from pathlib import Path
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from celluloid import Camera
from IPython.display import HTML

In [3]:
root = Path("/kaggle/input/3d-liver-and-liver-tumor-segmentation/08-3D-Liver-Tumor-Segmentation/08-3D-Liver-Tumor-Segmentation/Task03_Liver_rs/images/")
label = Path("/kaggle/input/3d-liver-and-liver-tumor-segmentation/08-3D-Liver-Tumor-Segmentation/08-3D-Liver-Tumor-Segmentation/Task03_Liver_rs/labels/")

In [4]:
def change_img_to_label_path(path):
    """
    Replaces imagesTr with labelsTr
    """
    parts = list(path.parts)
    parts[parts.index("images")] = "labels"
    return Path(*parts)

In [5]:
sample_path = list(root.glob("liver*"))[0]
sample_path_label = change_img_to_label_path(sample_path)

In [6]:
data = nib.load(sample_path)
label = nib.load(sample_path_label)

ct = data.get_fdata()
mask = label.get_fdata().astype(int)

In [7]:
nib.aff2axcodes(data.affine)

('R', 'A', 'S')

In [8]:
fig = plt.figure()
camera = Camera(fig) 

for i in range(ct.shape[2]):  # Axial view
    plt.imshow(ct[:,:,i], cmap="bone")
    mask_ = np.ma.masked_where(mask[:,:,i]==0, mask[:,:,i])
    plt.imshow(mask_, alpha=0.5)
    plt.axis("off")
    camera.snap()  # Store the current slice
plt.tight_layout()
animation = camera.animate()

<IPython.core.display.Javascript object>

In [9]:
HTML(animation.to_html5_video())

In [10]:
%matplotlib notebook
from pathlib import Path
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
from celluloid import Camera
from IPython.display import HTML
from concurrent.futures import ProcessPoolExecutor

def change_img_to_label_path(path):
    parts = list(path.parts)
    parts[parts.index("images")] = "labels"
    return Path(*parts)

def process_single_file(sample_path):
    sample_path_label = change_img_to_label_path(sample_path)
    
    data = nib.load(sample_path)
    label = nib.load(sample_path_label)
    
    ct = data.get_fdata()
    mask = label.get_fdata().astype(int)
    
    fig = plt.figure()
    camera = Camera(fig) 

    for i in range(ct.shape[2]):
        plt.imshow(ct[:,:,i], cmap="bone")
        mask_ = np.ma.masked_where(mask[:,:,i]==0, mask[:,:,i])
        plt.imshow(mask_, alpha=0.5)
        plt.axis("off")
        camera.snap()
    
    plt.tight_layout()
    animation = camera.animate()
    output_path = f"output/{sample_path.stem}.mp4"
    animation.save(output_path, writer="ffmpeg", fps=5)
    plt.close()

def main():
    root = Path("/kaggle/input/3d-liver-and-liver-tumor-segmentation/08-3D-Liver-Tumor-Segmentation/08-3D-Liver-Tumor-Segmentation/Task03_Liver_rs/images/")
    sample_paths = list(root.glob("liver*"))

    output_folder = Path("output")
    output_folder.mkdir(exist_ok=True)

    with ProcessPoolExecutor() as executor:
        executor.map(process_single_file, sample_paths)

if __name__ == "__main__":
    main()

In [11]:
from IPython.display import HTML
from pathlib import Path

def generate_video_grid(video_paths, columns=3):
    video_html = ""
    
    # Sort video paths based on the numerical part of the file name
    video_paths_sorted = sorted(video_paths, key=lambda x: int(Path(x).stem.split('_')[-1]))

    # Define the CSS styles
    video_style = 'style="flex: 1 1 30%; text-align:center; margin: 0 10px 5px 0;"'
    container_style = 'style="display:flex; flex-wrap:wrap; justify-content: space-around; padding: 10px;"'

    # Generate HTML for each video
    for i, video_path in enumerate(video_paths_sorted, start=1):
        file_name = Path(video_path).stem
        video_html += f'<div {video_style}>'
        video_html += f'    <div style="margin-bottom: 2px;"><b>{file_name}</b></div>'
        video_html += f'    <video width="275" height="275" controls><source src="{video_path}" type="video/mp4"></video>'
        video_html += f'</div>'
        
        # Check if a new line needs to be started
        if i % columns == 0:
            video_html += '<div style="flex-basis: 100%;"></div>'

    return f'<div {container_style}>{video_html}</div>'

# Specify the output folder
output_folder = Path("output")
video_paths = [str(video_path) for video_path in output_folder.glob("*.mp4")]

# Display the HTML using IPython.display
HTML(generate_video_grid(video_paths, columns=3))
